In [1]:
import os
from pathlib import Path


os.chdir(Path.cwd().parent)

In [2]:
import numpy as np

from src.rayleigh_quotient import rayleigh_quotient, rayleigh_quotient_naive
from src.utils.linalg import generate_normalized_vector, generate_supersymmetric_tensor
from src.utils.testing import timer

In [3]:
def test_rayleigh_quotient_base(A: np.ndarray,
                                 x: np.ndarray,
                                 use_timer: bool = False,
                                 verbose: bool = False) -> tuple[float, float] | None:
    """
    Test Rayleigh quotient computation with for loops and with numpy.

    :param A: supersymmetric tensor of shape (n x n x ... x n) of order k
    :param X: normalized vector of shape (n,)
    :param use_timer: whether to measure duration of computations
    :param verbose: whether to print more info

    :return: durations of naive and numpy approaches if |use_timer| is set
    """
    if verbose:
        print(f"testing Rayligh quotient computation with parameters:\n"
              f"tensor a of shape {A.shape}\na = {A}\n"
              f"vector x = {x}")

    rq_naive, duration_naive = timer(rayleigh_quotient_naive, A, x)
    rq_numpy, duration_numpy = timer(rayleigh_quotient, A, x)

    assert np.isclose(rq_naive, rq_numpy, atol=1e-6), f"{rq_naive=}\n{rq_numpy=}"

    if use_timer:
        return duration_naive, duration_numpy

In [4]:
def test_rayleigh_quotient_stress() -> None:
    n = 6
    k = 6
    for _ in range(10):
        A = generate_supersymmetric_tensor(n, k)
        x = generate_normalized_vector(n)
        test_rayleigh_quotient_base(A, x)

def test_rayleigh_quotient_stress_timer() -> None:
    n = 6
    k = 6
    durations_naive = []
    durations_numpy = []
    for _ in range(10):
        A = generate_supersymmetric_tensor(n, k)
        x = generate_normalized_vector(n)
        duration_naive, duration_numpy = test_rayleigh_quotient_base(A, x, use_timer=True)
        durations_naive.append(duration_naive)
        durations_numpy.append(duration_numpy)

    print(f"naive estimation duration: {np.mean(durations_naive)}")
    print(f"numpy estimation duration: {np.mean(durations_numpy)}")


In [5]:
test_rayleigh_quotient_stress()
test_rayleigh_quotient_stress_timer()

naive estimation duration: 0.3008802652359009
numpy estimation duration: 0.0001299142837524414
